In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import csv
import glob
import imageio.v3 as iio
import matplotlib
import numpy as np
import os
import pandas as pd
import pickle

In [ ]:
ROOT_FILE_PATH = "/content/drive/My Drive/2023-Machine-Learning-Dataset";
SUBJ_LIST = ["subj01", "subj02", "subj03", "subj04", "subj05", "subj06", "subj07", "subj08"];
AREAS = ["prf-visualrois", "floc-bodies", "floc-faces", "floc-places", "floc-words", "streams", "all-vertices"];

def ReadNpyFile(folderdir):
  retdic = {};
  for path in glob.glob(folderdir + "/*.npy"):
    filename = os.path.splitext(path)[0].split("/")[-1];
    retdic[filename] = np.load(path, allow_pickle = True);
  return retdic;

def ReadFmriFile(folderdir):
  retdic = {};
  for path in glob.glob(folderdir + "/*.npy"):
    filename = os.path.splitext(path)[0].split("/")[-1];
    retdic[filename[0]] = np.load(path, allow_pickle = True);
  return retdic;

def ReadMaskFile(folderdir):
  retdic = {};
  for path in glob.glob(folderdir + "/*.npy"):
    filename = os.path.splitext(path)[0].split("/")[-1];
    for area in AREAS:
      if area in filename:
        filename = "".join(i.strip("_") for i in filename.split(area));
        break;
    if area not in retdic:
      retdic[area] = {};
    retdic[area][filename] = np.load(path, allow_pickle = True) if filename[0] != "m" else np.load(path, allow_pickle = True).item();
  return retdic;  

def ReadPngFile(folderdir):
  retdic = {};
  for path in glob.glob(folderdir + "/*.png"):
    filename = os.path.splitext(path)[0].split("/")[-1];
    retdic[path.split("/")[-1]] = iio.imread(path);
  return retdic;


In [ ]:
def read_subject(subject):
  retdic = {};
  retdic["training_img"] = ReadPngFile(os.path.join(ROOT_FILE_PATH, subject, "training_split", "training_images"))
  retdic["training_fmri"] = ReadNpyFile(os.path.join(ROOT_FILE_PATH, subject, "training_split", "training_fmri"));
  retdic["test_fmri"] = ReadNpyFile(os.path.join(ROOT_FILE_PATH, subject, "test_split", "test_fmri"));
  retdic["roi_masks"] = ReadNpyFile(os.path.join(ROOT_FILE_PATH, subject, "roi_masks"));
  return retdic;

In [ ]:
class ReadObject:
  subject = None;     # string
  images = None;      # dict(string -> ndarray, uint8)
  fmri = None;        # dict(string -> ndarray, float32)
  masks = None;       # dict(string "AREA" -> (dict(int -> string), string "HEMISPHERE, " -> ndarray))
  image_data = None;  # dataframe

  def load(self, subject, mode = "train"):
    #Mode = "train" | "test"
    if(subject != self.subject):
      self.subject = subject;
      self.masks = ReadMaskFile(os.path.join(ROOT_FILE_PATH, subject , "roi_masks"));
      self.image_data = pd.read_csv(os.path.join(ROOT_FILE_PATH, "image_infos", f"{subject}_infos_train.csv"));
    if(mode == "train"):
      self.images = ReadPngFile(os.path.join(ROOT_FILE_PATH, subject, "training_split", "training_images"));
      self.fmri = ReadNpyFile(os.path.join(ROOT_FILE_PATH, subject, "training_split", "training_fmri"));
    elif(mode == "test"):
      self.images = None;
      self.fmri = ReadNpyFile(os.path.join(ROOT_FILE_PATH, subject, "test_split", "test_fmri"));

  def get_hsv_image(self, image_name):
    return matplotlib.colors.rgb_to_hsv(self.images[image_name]);

  def sample_data(self, sample_number = 100):
    if(self.images == None):
      return None;
    #TODO: Sample Data ICARUS PLEASE DO THIS

In [ ]:
DATA = ReadObject(); 
DATA.load(SUBJ_LIST[0]); #Requires 5GB approx.